In [ ]:
# Merge all 3 datasets
merged_data = (
    unvotes
    .merge(roll_calls_no_amend_para, on='rcid', how='left')
    .merge(issues, on='rcid', how='left')
)
# Keep relevant columns
merged_data = merged_data[['rcid', 'country', 'country_code', 'vote', 'date', 'short_name']]

# Filter for Cold War Period
merged_data = merged_data[merged_data['date'].between('1947-01-01', '1991-12-31')].copy()

# Filter for Arms & Nuclear issues only
merged_data = merged_data[merged_data['short_name'].isin(['Arms', 'Nuclear'])]

In [ ]:
# Create reference vote columns for US and Russia
us_votes = merged_data[merged_data['country'] == 'United States'][['rcid', 'vote']].rename(columns={'vote': 'us_vote'})
ru_votes = merged_data[merged_data['country'].isin(['Russia', 'Soviet Union'])][['rcid', 'vote']].rename(columns={'vote': 'ru_vote'})

# Merge the reference votes for US and Russia
alignment = (
    merged_data
    .merge(us_votes, on='rcid', how='left')
    .merge(ru_votes, on='rcid', how='left')
)
# Remove NA values
alignment = alignment.dropna(subset=['us_vote', 'ru_vote'])

# Compute alignment with each session
alignment['align_us'] = (alignment['vote'] == alignment['us_vote']).astype(int)
alignment['align_ru'] = (alignment['vote'] == alignment['ru_vote']).astype(int)

# Compute the alignment score
alignment_score = (
    alignment.groupby(['country', 'country_code'], as_index=False)[['align_us', 'align_ru']]
    .mean()
    .sort_values('align_us', ascending=False)
)


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import geopandas as gpd
import json
import plotly.io as pio

# Keep Visualisations Interactive in HTML
pio.renderers.default = 'vscode' 
# pio.renderers.default = 'iframe'

In [ ]:
# Load a high-resolution world shapefile via a link
world = gpd.read_file("https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_0_countries.zip")

# Merge with Alignment Score
world = world.merge(alignment_score, left_on="ISO_A2", right_on="country_code", how="left")

# Convert to GeoJSON for Plotly
geojson_data = json.loads(world.to_json())

# Create 1x2 subplot layout
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("<b>Alignment with the United States<b>", "<b>Alignment with the USSR / Russia<b>"),
    specs=[[{"type": "choropleth"}, {"type": "choropleth"}]]
)

# Positioning subplot titles 
for annotation in fig['layout']['annotations']:
    annotation['y'] -= 0.17

# Base configuration for consistent colorbars as legends
colorbar_style = dict(
    tickvals=[0, 0.25, 0.5, 0.75, 1],
    ticktext=['Low', '', 'Neutral', '', 'High'],
    tickfont=dict(size=12, color='#222', family="Lato, Open Sans, sans-serif"),
    outlinecolor='black',
    outlinewidth=1,
    thickness=14,
    lenmode='pixels',
    len=300,
    yanchor='middle',
    y=0.5,
)

# Left Subplot: US Alignment
fig.add_trace(
    go.Choropleth(
        geojson=geojson_data,
        featureidkey='properties.ISO_A2',
        locations=alignment_score['country_code'],
        z=alignment_score['align_us'],
        colorscale='YlGnBu',
        zmin=0, zmax=1,
        hovertext=alignment_score['country'],
        hovertemplate="<b>%{hovertext}</b><br>US Alignment: %{z:.2f}<extra></extra>",
        colorbar=dict(
            **colorbar_style,
            x=0.46, 
            xanchor="center"
        )
    ),
    row=1, col=1
)

# Right Subplot: USSR Alignment 
fig.add_trace(
    go.Choropleth(
        geojson=geojson_data,
        featureidkey='properties.ISO_A2',
        locations=alignment_score['country_code'],
        z=alignment_score['align_ru'],
        colorscale='OrRd',
        zmin=0, zmax=1,
        hovertext=alignment_score['country'],
        hovertemplate="<b>%{hovertext}</b><br>USSR Alignment: %{z:.2f}<extra></extra>",
        colorbar=dict(
            **colorbar_style,
            x=1.01, 
            xanchor="center"
        )
    ),
    row=1, col=2
)

# Global layout settings
fig.update_layout(
    title=dict(
        text="<b>Global Alignment on Arms & Nuclear Issues during the Cold War (1947–1991)</b>",
        x=0.5, y=0.85,
        font=dict(size=22, family="Lato, Open Sans, sans-serif", color="#111")
    ),
    font=dict(
        family="Open Sans, Lato, sans-serif",
        size=13,
        color="#222"
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    height=620,
    margin=dict(l=0, r=0, t=90, b=20)
)

# Shared geo styling
for i in [1, 2]:
    fig.update_geos(
        showcountries=True,
        showframe=False,
        showcoastlines=True,
        coastlinecolor="#E0E0E0",
        showland=True,
        landcolor="#F7F7F7",
        showocean=True,
        oceancolor="#A8CBE2",
        projection_type="natural earth",
        bgcolor='white',
        row=1, col=i,
        projection_scale=0.9
    )

# Note for missing data 
fig.add_annotation(
    text="⬜️ White regions indicate missing or unavailable voting data.",
    x=0.53, y=0.08,
    xref="paper", yref="paper",
    showarrow=False,
    font=dict(size=15, color="gray", family="Lato, Open Sans, sans-serif"),
    align="center"
)

fig.show()